In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/AItom/safety_embedding_model

/content/drive/MyDrive/AItom/safety_embedding_model


In [3]:
import os
import sys
import argparse
from pathlib import Path
from typing import List, Tuple, Optional
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score, confusion_matrix, classification_report
)
from tqdm import tqdm

from utils.cas_to_formula import cas_to_formula
from utils.multi_property_embedding import get_combined_embedding, DEFAULT_PROPERTIES


In [4]:
class RiskDataset(Dataset):
    """위험성 분류를 위한 데이터셋"""

    def __init__(self, formulas: List[str] = None, labels: np.ndarray = None,
                 embeddings: torch.Tensor = None, use_embeddings: bool = False):
        """
        Parameters
        ----------
        formulas : List[str], optional
            화학식 리스트 (use_embeddings=False일 때 사용)
        labels : np.ndarray, optional
            라벨 배열 (N,) - 0 또는 1
        embeddings : torch.Tensor, optional
            임베딩 벡터 텐서 (N, embedding_dim) (use_embeddings=True일 때 사용)
        use_embeddings : bool
            True면 embeddings를 직접 사용, False면 formulas에서 임베딩 추출
        """
        self.use_embeddings = use_embeddings

        if use_embeddings:
            # 임베딩을 직접 사용하는 경우
            if embeddings is None or labels is None:
                raise ValueError("use_embeddings=True일 때 embeddings와 labels가 필요합니다.")
            self.embeddings = embeddings
            self.labels = torch.LongTensor(labels) if isinstance(labels, np.ndarray) else labels
            self.formulas = None  # 필요시 저장 가능
        else:
            # 기존 방식: formulas에서 임베딩 추출
            if formulas is None or labels is None:
                raise ValueError("use_embeddings=False일 때 formulas와 labels가 필요합니다.")
            self.formulas = formulas
            self.labels = torch.LongTensor(labels) if isinstance(labels, np.ndarray) else labels
            self.embeddings = None

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        if self.use_embeddings:
            return self.embeddings[idx], self.labels[idx]
        else:
            return self.formulas[idx], self.labels[idx]

In [5]:
class RiskClassifier(nn.Module):
    """위험성 이진 분류를 위한 MLP 모델"""

    def __init__(
        self,
        properties: List[str] = None,
        input_dim: int = 84,
        hidden_dims: List[int] = [256, 128, 64, 32],
        dropout_rate: float = 0,
        compute_device: Optional[str] = None,
        use_embeddings: bool = False,
    ):
        """
        Parameters
        ----------
        properties : List[str]
            사용할 property 리스트 (임베딩 차원 결정)
        hidden_dims : List[int]
            은닉층 차원 리스트
        dropout_rate : float
            Dropout 비율
        compute_device : str, optional
            계산 디바이스 (CrabNet 임베딩 추출용)
        """
        super(RiskClassifier, self).__init__()

        self.properties = properties
        self.compute_device = compute_device
        self.use_embeddings = use_embeddings
        self.input_dim = input_dim


        layers = []
        prev_dim = self.input_dim

        for hidden_dim in hidden_dims:
            layers.append(nn.Linear(prev_dim, hidden_dim))
            layers.append(nn.SiLU())
            layers.append(nn.BatchNorm1d(hidden_dim))
            layers.append(nn.Dropout(dropout_rate))
            prev_dim = hidden_dim

        # 출력층 (이진 분류)
        layers.append(nn.Linear(prev_dim, 2))

        self.network = nn.Sequential(*layers)

    def forward(self, inputs) -> torch.Tensor:
        """
        Parameters
        ----------
        inputs : torch.Tensor or List[str]
            - torch.Tensor: 임베딩 벡터 (batch_size, embedding_dim) - use_embeddings=True일 때
            - List[str]: 화학식 리스트 (배치) - use_embeddings=False일 때

        Returns
        -------
        outputs : torch.Tensor
            분류 로짓 (batch_size, 2)
        valid_indices : torch.Tensor (optional)
            유효한 샘플의 인덱스 (use_embeddings=False일 때만 반환)
        """
        # 임베딩을 직접 받는 경우
        if isinstance(inputs, torch.Tensor):
            x = inputs
            return self.network(x)

        # Formula 리스트를 받는 경우 (기존 방식)
        formulas = inputs
        embeddings = []
        valid_indices = []
        failed_formulas = []
        with torch.no_grad():  # CrabNet 임베딩 추출 시 gradient 비활성화
            for idx, formula in enumerate(formulas):
                try:
                    emb = get_combined_embedding(
                        formula=formula,
                        properties=self.properties,
                        compute_device=self.compute_device,
                        verbose=False,
                    )
                    # 1D 벡터를 명시적으로 확인하고 차원 보장
                    if emb.dim() == 1:
                        embeddings.append(emb)
                    else:
                        # 이미 2D인 경우 flatten
                        embeddings.append(emb.flatten())
                    valid_indices.append(idx)
                except Exception as e:
                    # 임베딩 추출 실패 시 formula 기록
                    failed_formulas.append((formula, str(e)))
                    print(f"[ERROR] 임베딩 추출 실패 - formula: {formula}, error: {e}")

        # 실패한 formula가 있으면 경고
        if failed_formulas:
            print(f"[WARNING] {len(failed_formulas)}개 formula의 임베딩 추출 실패:")
            for formula, error in failed_formulas:
                print(f"  - {formula}: {error}")

        # 유효한 임베딩이 없으면 에러
        if len(embeddings) == 0:
            raise RuntimeError(
                f"모든 formula의 임베딩 추출 실패. 배치 크기: {len(formulas)}, "
                f"실패한 formula: {[f[0] for f in failed_formulas]}"
            )

        # 배치로 스택: (batch_size, embedding_dim) 형태로 변환
        x = torch.stack(embeddings)  # (valid_batch_size, embedding_dim)

        # 임베딩 수와 원래 formula 수가 다르면 경고
        if len(embeddings) != len(formulas):
            print(f"[WARNING] 배치 크기 불일치: 원래 {len(formulas)}개, 성공 {len(embeddings)}개")
            print(f"  실패한 formula: {[f[0] for f in failed_formulas]}")

        # 첫 번째 forward에서 실제 임베딩 차원 확인 및 MLP 재구성
        if not self._input_dim_initialized:
            actual_dim = x.shape[1]
            if actual_dim != self.input_dim:
                self.input_dim = actual_dim
                if hasattr(self, 'network') and len(self.network) > 0:
                    first_layer = self.network[0]
                    if isinstance(first_layer, nn.Linear) and first_layer.in_features != actual_dim:
                        print(f"Warning: Input dimension mismatch. Expected {first_layer.in_features}, got {actual_dim}")
                        print(f"Please ensure the model is initialized with correct input_dim or recreate the model.")
            self._input_dim_initialized = True

        x = x.detach()  # CrabNet gradient 완전 차단

        # MLP 통과 (MLP만 학습됨)
        outputs = self.network(x)

        # 유효한 인덱스를 텐서로 변환
        valid_indices_tensor = torch.tensor(valid_indices, dtype=torch.long)

        return outputs, valid_indices_tensor

In [6]:
def train_epoch(model, dataloader, criterion, optimizer, device, epoch=None, use_embeddings=False):
    """한 에폭 훈련"""
    model.train()
    total_loss = 0.0
    all_preds = []
    all_labels = []
    processed_batches = 0

    # tqdm으로 진행 바 표시
    desc = f"Epoch {epoch}" if epoch is not None else "Training"
    pbar = tqdm(dataloader, desc=desc, leave=False)

    for batch_data in pbar:
        if use_embeddings:
            # embeddings를 직접 사용하는 경우
            embeddings, labels = batch_data
            labels = labels.to(device)
            embeddings = embeddings.to(device)

            optimizer.zero_grad()
            try:
                outputs = model(embeddings)
                outputs = outputs.to(device)

                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                total_loss += loss.item()
                processed_batches += 1
                _, preds = torch.max(outputs, 1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

                # 진행 바 업데이트
                current_loss = total_loss / processed_batches
                current_acc = accuracy_score(all_labels, all_preds) if len(all_labels) > 0 else 0.0
                pbar.set_postfix({
                    'loss': f'{current_loss:.4f}',
                    'acc': f'{current_acc:.4f}',
                    'processed': processed_batches
                })
            except Exception as e:
                pbar.set_postfix({'status': f'error: {str(e)[:30]}'})
                print(f"[ERROR] 모델 forward 실패")
                print(f"  에러: {e}")
                continue
        else:
            # 기존 방식: formulas에서 임베딩 추출
            formulas, labels = batch_data
            labels = labels.to(device)

            optimizer.zero_grad()
            # 모델이 formula 리스트를 받아서 임베딩 추출 후 분류
            try:
                outputs, valid_indices = model(formulas)
                outputs = outputs.to(device)
                valid_indices = valid_indices.to(device)

                # 유효한 샘플의 labels만 필터링
                valid_labels = labels[valid_indices]

                # 유효한 샘플이 없으면 스킵
                if len(valid_labels) == 0:
                    pbar.set_postfix({'status': 'skipped (no valid samples)'})
                    continue

                loss = criterion(outputs, valid_labels)
                loss.backward()
                optimizer.step()

                total_loss += loss.item()
                processed_batches += 1
                _, preds = torch.max(outputs, 1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(valid_labels.cpu().numpy())

                # 진행 바 업데이트 (현재 loss와 accuracy 표시)
                current_loss = total_loss / processed_batches
                current_acc = accuracy_score(all_labels, all_preds) if len(all_labels) > 0 else 0.0
                pbar.set_postfix({
                    'loss': f'{current_loss:.4f}',
                    'acc': f'{current_acc:.4f}',
                    'processed': processed_batches
                })
            except Exception as e:
                pbar.set_postfix({'status': f'error: {str(e)[:30]}'})
                print(f"[ERROR] 모델 forward 실패 - formula들: {formulas}")
                print(f"  에러: {e}")
                continue

    pbar.close()

    if processed_batches == 0:
        avg_loss = 0.0
        accuracy = 0.0
    else:
        avg_loss = total_loss / processed_batches
        accuracy = accuracy_score(all_labels, all_preds) if len(all_labels) > 0 else 0.0

    return avg_loss, accuracy


def evaluate(model, dataloader, criterion, device, use_embeddings=False):
    """모델 평가"""
    model.eval()
    total_loss = 0.0
    all_preds = []
    all_probs = []
    all_labels = []
    processed_batches = 0

    with torch.no_grad():
        for batch_data in dataloader:
            if use_embeddings:
                # embeddings를 직접 사용하는 경우
                embeddings, labels = batch_data
                labels = labels.to(device)
                embeddings = embeddings.to(device)

                try:
                    outputs = model(embeddings)
                    outputs = outputs.to(device)

                    loss = criterion(outputs, labels)

                    total_loss += loss.item()
                    processed_batches += 1
                    probs = torch.softmax(outputs, dim=1)
                    _, preds = torch.max(outputs, 1)

                    all_preds.extend(preds.cpu().numpy())
                    all_probs.extend(probs[:, 1].cpu().numpy())
                    all_labels.extend(labels.cpu().numpy())
                except Exception as e:
                    print(f"[ERROR] 모델 forward 실패")
                    print(f"  에러: {e}")
                    continue
            else:
                # 기존 방식: formulas에서 임베딩 추출
                formulas, labels = batch_data
                labels = labels.to(device)

                # 모델이 formula 리스트를 받아서 임베딩 추출 후 분류
                try:
                    outputs, valid_indices = model(formulas)
                    outputs = outputs.to(device)
                    valid_indices = valid_indices.to(device)

                    # 유효한 샘플의 labels만 필터링
                    valid_labels = labels[valid_indices]

                    # 유효한 샘플이 없으면 스킵
                    if len(valid_labels) == 0:
                        print(f"[WARNING] 배치 내 유효한 샘플이 없음 - formula들: {formulas}")
                        continue

                    loss = criterion(outputs, valid_labels)

                    total_loss += loss.item()
                    processed_batches += 1
                    probs = torch.softmax(outputs, dim=1)
                    _, preds = torch.max(outputs, 1)

                    all_preds.extend(preds.cpu().numpy())
                    all_probs.extend(probs[:, 1].cpu().numpy())  # 위험 클래스 확률
                    all_labels.extend(valid_labels.cpu().numpy())
                except Exception as e:
                    print(f"[ERROR] 모델 forward 실패 - formula들: {formulas}")
                    print(f"  에러: {e}")
                    continue

    if processed_batches == 0:
        avg_loss = 0.0
        accuracy = 0.0
        precision = 0.0
        recall = 0.0
        f1 = 0.0
        auc = 0.0
    else:
        avg_loss = total_loss / processed_batches
        if len(all_labels) == 0:
            accuracy = 0.0
            precision = 0.0
            recall = 0.0
            f1 = 0.0
            auc = 0.0
        else:
            accuracy = accuracy_score(all_labels, all_preds)
            precision = precision_score(all_labels, all_preds, zero_division=0)
            recall = recall_score(all_labels, all_preds, zero_division=0)
            f1 = f1_score(all_labels, all_preds, zero_division=0)
            try:
                auc = roc_auc_score(all_labels, all_probs)
            except ValueError:
                auc = 0.0

    return {
        'loss': avg_loss,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'auc': auc,
        'predictions': all_preds,
        'probabilities': all_probs,
        'labels': all_labels,
    }

In [7]:
# 랜덤 시드 설정
torch.manual_seed(45)
np.random.seed(45)

# 디바이스 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"사용 디바이스: {device}")

사용 디바이스: cpu


In [8]:
embeddings_path1 = '/content/drive/MyDrive/AItom/safety_embedding_model/training_data/embeddings4.pt'
embeddings_path2 = '/content/drive/MyDrive/AItom/safety_embedding_model/training_data/embeddings5.pt'
embeddings_path3 = '/content/drive/MyDrive/AItom/safety_embedding_model/training_data/embeddings6.pt'
embeddings_path4 = '/content/drive/MyDrive/AItom/safety_embedding_model/training_data/embeddings7.pt'
embeddings_path5 = '/content/drive/MyDrive/AItom/safety_embedding_model/training_data/embeddings8.pt'
embeddings_path6 = '/content/drive/MyDrive/AItom/safety_embedding_model/training_data/embeddings9.pt'

embeddings_data1 = torch.load(embeddings_path1)
embeddings_data2 = torch.load(embeddings_path2)
embeddings_data3 = torch.load(embeddings_path3)
embeddings_data4 = torch.load(embeddings_path4)
embeddings_data5 = torch.load(embeddings_path5)
embeddings_data6 = torch.load(embeddings_path6)

embeddings1 = embeddings_data1['embeddings']
labels1 = embeddings_data1['labels']  # (N,)
formulas = embeddings_data1.get('formulas', [])  # List[str] (optional)
properties = embeddings_data1.get('properties', DEFAULT_PROPERTIES)

embeddings2 = embeddings_data2['embeddings']
labels2 = embeddings_data2['labels']  # (N,)
formulas2 = embeddings_data2.get('formulas', [])  # List[str] (optional)

embeddings3 = embeddings_data3['embeddings']
labels3 = embeddings_data3['labels']  # (N,)
formulas3 = embeddings_data3.get('formulas', [])  # List[str] (optional)

embeddings4 = embeddings_data4['embeddings']
labels4 = embeddings_data4['labels']  # (N,)
formulas4 = embeddings_data4.get('formulas', [])  # List[str] (optional)

embeddings5 = embeddings_data5['embeddings']
labels5 = embeddings_data5['labels']  # (N,)
formulas5 = embeddings_data5.get('formulas', [])  # List[str] (optional)

embeddings6 = embeddings_data6['embeddings']
labels6 = embeddings_data6['labels']  # (N,)
formulas6 = embeddings_data6.get('formulas', [])  # List[str] (optional)

embeddings = torch.cat(
    [embeddings1, embeddings2, embeddings3,
     embeddings4, embeddings5, embeddings6],
    dim=0
)

labels = torch.cat([labels1, labels2, labels3, labels4, labels5, labels6], dim=0)
formulas.extend(formulas2)
formulas.extend(formulas3)
formulas.extend(formulas4)
formulas.extend(formulas5)
formulas.extend(formulas6)



In [9]:


embedding_dim = embeddings.shape[1]

print(f"  - 샘플 수: {len(embeddings)}")
print(f"  - 임베딩 차원: {embedding_dim}")
print(f"  - Properties: {len(properties)}개")
print(f"  - 라벨 분포: 위험(1)={torch.sum(labels==1).item()}, 비위험(0)={torch.sum(labels==0).item()}")

# labels를 numpy로 변환
if isinstance(labels, torch.Tensor):
    labels_np = labels.numpy()
else:
    labels_np = np.array(labels)

test_size = 0.2
val_size = 0.2
seed = 100

# 2. 데이터 분할 (embeddings 사용)
indices = np.arange(len(embeddings))
X_train_idx, X_temp_idx, y_train, y_temp = train_test_split(
            indices, labels_np,
            test_size=test_size + val_size,
            random_state=seed,
            stratify=labels_np,
)

val_size_adjusted = val_size / (test_size + val_size)
X_val_idx, X_test_idx, y_val, y_test = train_test_split(
            X_temp_idx, y_temp,
            test_size=1 - val_size_adjusted,
            random_state=seed,
            stratify=y_temp,
)

print("\n데이터 분할:")
print(f"  - Train: {len(X_train_idx)} ({100*len(X_train_idx)/len(embeddings):.1f}%)")
print(f"  - Val: {len(X_val_idx)} ({100*len(X_val_idx)/len(embeddings):.1f}%)")
print(f"  - Test: {len(X_test_idx)} ({100*len(X_test_idx)/len(embeddings):.1f}%)")


  - 샘플 수: 3008
  - 임베딩 차원: 36
  - Properties: 12개
  - 라벨 분포: 위험(1)=1782, 비위험(0)=1226

데이터 분할:
  - Train: 1804 (60.0%)
  - Val: 602 (20.0%)
  - Test: 602 (20.0%)


In [10]:
batch_size = 512


train_dataset = RiskDataset(
            embeddings=embeddings[X_train_idx],
            labels=y_train,
            use_embeddings=True
)
val_dataset = RiskDataset(
            embeddings=embeddings[X_val_idx],
            labels=y_val,
            use_embeddings=True
)
test_dataset = RiskDataset(
            embeddings=embeddings[X_test_idx],
            labels=y_test,
            use_embeddings=True
)

model_input_dim = embedding_dim

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [11]:
model = RiskClassifier(
        input_dim=model_input_dim,
        compute_device=str(device),
        use_embeddings=True,
)

print("\n모델 구조:")
print(model)
# 디바이스로 이동
model = model.to(device)


모델 구조:
RiskClassifier(
  (network): Sequential(
    (0): Linear(in_features=36, out_features=256, bias=True)
    (1): SiLU()
    (2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0, inplace=False)
    (4): Linear(in_features=256, out_features=128, bias=True)
    (5): SiLU()
    (6): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): Dropout(p=0, inplace=False)
    (8): Linear(in_features=128, out_features=64, bias=True)
    (9): SiLU()
    (10): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): Dropout(p=0, inplace=False)
    (12): Linear(in_features=64, out_features=32, bias=True)
    (13): SiLU()
    (14): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (15): Dropout(p=0, inplace=False)
    (16): Linear(in_features=32, out_features=2, bias=True)
  )
)


In [12]:
#lr = 0.001

lr = 0.01



criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-9)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=2
)

In [13]:
print("\n" + "="*60)
print("훈련 시작")
print("="*60)

best_val_f1 = 0.0
best_epoch = 0
train_history = []

use_embeddings = True
epochs = 500

hidden_dims = [256, 128, 64, 32]


save_dir = '/content/drive/MyDrive/AItom/safety_embedding_model/training_results'

for epoch in range(epochs):
    # 훈련
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device, epoch=epoch+1, use_embeddings=use_embeddings)

    # 검증
    val_metrics = evaluate(model, val_loader, criterion, device, use_embeddings=use_embeddings)

    # 학습률 스케줄러 업데이트
    scheduler.step(val_metrics['loss'])

    # 기록
    train_history.append({
            'epoch': epoch + 1,
            'train_loss': train_loss,
            'train_acc': train_acc,
            'val_loss': val_metrics['loss'],
            'val_acc': val_metrics['accuracy'],
            'val_f1': val_metrics['f1'],
            'val_auc': val_metrics['auc'],
    })

    # 최고 성능 모델 저장
    if val_metrics['f1'] > best_val_f1:
        best_val_f1 = val_metrics['f1']
        best_epoch = epoch + 1

        # 모델 저장
        os.makedirs(save_dir, exist_ok=True)
        torch.save({
                'epoch': epoch + 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_f1': best_val_f1,
                'properties': DEFAULT_PROPERTIES,
                'hidden_dims': hidden_dims,
        }, os.path.join(save_dir, 'best_model6.pth'))

    # 진행 상황 출력
    if (epoch + 1) % 5 == 0 or epoch == 0:
        print(f"Epoch {epoch+1:3d}/{epochs} | "
                  f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | "
                  f"Val Loss: {val_metrics['loss']:.4f} | Val Acc: {val_metrics['accuracy']:.4f} | "
                  f"Val F1: {val_metrics['f1']:.4f} | Val AUC: {val_metrics['auc']:.4f}")

print("\n최고 성능: Epoch {best_epoch}, Val F1: {best_val_f1:.4f}")



훈련 시작


Epoch   1/500 | Train Loss: 0.7025 | Train Acc: 0.5815 | Val Loss: 0.7032 | Val Acc: 0.4884 | Val F1: 0.2524 | Val AUC: 0.6895


Epoch   5/500 | Train Loss: 0.5046 | Train Acc: 0.7605 | Val Loss: 0.6116 | Val Acc: 0.6827 | Val F1: 0.7766 | Val AUC: 0.7702


Epoch  10/500 | Train Loss: 0.4426 | Train Acc: 0.7894 | Val Loss: 0.7363 | Val Acc: 0.6312 | Val F1: 0.7477 | Val AUC: 0.7115


Epoch  15/500 | Train Loss: 0.3688 | Train Acc: 0.8331 | Val Loss: 0.5206 | Val Acc: 0.7508 | Val F1: 0.8021 | Val AUC: 0.8063


...

Epoch 480/500 | Train Loss: 0.2911 | Train Acc: 0.8731 | Val Loss: 0.6140 | Val Acc: 0.7641 | Val F1: 0.7989 | Val AUC: 0.8095


Epoch 485/500 | Train Loss: 0.2855 | Train Acc: 0.8675 | Val Loss: 0.6118 | Val Acc: 0.7674 | Val F1: 0.8017 | Val AUC: 0.8096


Epoch 490/500 | Train Loss: 0.2862 | Train Acc: 0.8731 | Val Loss: 0.6158 | Val Acc: 0.7658 | Val F1: 0.8000 | Val AUC: 0.8093


Epoch 495/500 | Train Loss: 0.2953 | Train Acc: 0.8647 | Val Loss: 0.6140 | Val Acc: 0.7658 | Val F1: 0.8011 | Val AUC: 0.8090


Epoch 500/500 | Train Loss: 0.2869 | Train Acc: 0.8703 | Val Loss: 0.6136 | Val Acc: 0.7691 | Val F1: 0.8039 | Val AUC: 0.8096

최고 성능: Epoch {best_epoch}, Val F1: {best_val_f1:.4f}


In [14]:

print("\n" + "="*60)
print("테스트 세트 평가")
print("="*60)

# 최고 모델 로드
checkpoint = torch.load(os.path.join(save_dir, 'best_model6.pth'))
model.load_state_dict(checkpoint['model_state_dict'])

test_metrics = evaluate(model, test_loader, criterion, device, use_embeddings=use_embeddings)

print("\n테스트 세트 성능:")
print(f"  - Accuracy: {test_metrics['accuracy']:.4f}")
print(f"  - Precision: {test_metrics['precision']:.4f}")
print(f"  - Recall: {test_metrics['recall']:.4f}")
print(f"  - F1 Score: {test_metrics['f1']:.4f}")
print(f"  - AUC-ROC: {test_metrics['auc']:.4f}")

# Confusion Matrix
cm = confusion_matrix(test_metrics['labels'], test_metrics['predictions'])
print("\nConfusion Matrix:")
print("                Predicted")
print("              Safe  Risk")
print(f"Actual Safe    {cm[0,0]:4d}  {cm[0,1]:4d}")
print(f"       Risk    {cm[1,0]:4d}  {cm[1,1]:4d}")

# Classification Report
print("\nClassification Report:")
print(classification_report(
        test_metrics['labels'],
        test_metrics['predictions'],
        target_names=['Safe', 'Risk']
))

# 히스토리 저장
history_df = pd.DataFrame(train_history)
history_df.to_csv(os.path.join(save_dir, 'training_history.csv'), index=False)
print(f"\n훈련 히스토리 저장: {os.path.join(save_dir, 'training_history.csv')}")

print("\n훈련 완료!")


테스트 세트 평가

테스트 세트 성능:
  - Accuracy: 0.8056
  - Precision: 0.8504
  - Recall: 0.8146
  - F1 Score: 0.8321
  - AUC-ROC: 0.8609

Confusion Matrix:
                Predicted
              Safe  Risk
Actual Safe     195    51
       Risk      66   290

Classification Report:
              precision    recall  f1-score   support

        Safe       0.75      0.79      0.77       246
        Risk       0.85      0.81      0.83       356

    accuracy                           0.81       602
   macro avg       0.80      0.80      0.80       602
weighted avg       0.81      0.81      0.81       602


훈련 히스토리 저장: /content/drive/MyDrive/AItom/safety_embedding_model/training_results/training_history.csv

훈련 완료!
